In [4]:
import dash
import pandas as pd
import plotly.express as px
import threading
from dash import dcc, html
from dash.dependencies import Input, Output
!pip install nbformat
import plotly.io as pio
!pip install kaleido==0.2.1
!pip install --upgrade kaleido
!pip install --upgrade plotly kaleido

  Using cached kaleido-0.2.1-py2.py3-none-macosx_11_0_arm64.whl.metadata (15 kB)
Using cached kaleido-0.2.1-py2.py3-none-macosx_11_0_arm64.whl (85.8 MB)
  Attempting uninstall: kaleido
    Found existing installation: kaleido 1.1.0
    Uninstalling kaleido-1.1.0:
      Successfully uninstalled kaleido-1.1.0
  Using cached kaleido-1.1.0-py3-none-any.whl.metadata (5.6 kB)
Using cached kaleido-1.1.0-py3-none-any.whl (66 kB)
  Attempting uninstall: kaleido
    Found existing installation: kaleido 0.2.1
    Uninstalling kaleido-0.2.1:
      Successfully uninstalled kaleido-0.2.1


In [5]:
df = pd.read_csv('cleaned_trojmiasto.csv')
df.head()

,url,date_posted,photos,locality,street,property_type,rooms,area_m2,owner_direct,price_total_zl,price_per_m2_zl,city,price_total_zl_cleaned,price_per_m2_zl_cleaned,area_per_room
0,https://adresowo.pl/o/mieszkanie-gdansk-jasien...,3 dni temu,12.0,Gdańsk Jasień,ul. Kazimierza Leskiego,Apartment,2.0,41.0,Bez pośredników,639000,15582.0,Gdańsk,639000.0,15582.0,20.500000
1,https://adresowo.pl/o/mieszkanie-gdynia-srodmi...,dzisiajnowe,16.0,Gdynia Śródmieście,ul. A. Hryniewickiego,Apartment,3.0,80.0,Bez pośredników,2800000,35000.0,Gdynia,2800000.0,35000.0,26.666667
2,https://adresowo.pl/o/mieszkanie-gdansk-wrzesz...,dzisiajnowe,7.0,Gdańsk Wrzeszcz,NaN,Apartment,1.0,27.0,Bez pośredników,465000,17222.0,Gdańsk,465000.0,17222.0,27.000000
3,https://adresowo.pl/o/mieszkanie-gdansk-morena...,dzisiajnowe,10.0,Gdańsk Morena,ul. Warneńska,Apartment,2.0,47.0,Oferta biura nieruchomości,569000,12106.0,Gdańsk,569000.0,12106.0,23.500000
4,https://adresowo.pl/o/mieszkanie-gdansk-ul-pow...,dzisiajnowe,17.0,Gdańsk,ul. Powstania Listopadowego,Apartment,2.0,47.0,Oferta biura nieruchomości,550000,11677.0,Gdańsk,550000.0,11677.0,23.500000


In [6]:
# Oblicz podstawowe statystyki (średnia, mediana, rozkład).
cols = ['area_m2', 'price_total_zl_cleaned', 'price_per_m2_zl_cleaned']
stats = df[cols].describe()
# Dodaję medianę osobno (bo nie jest w describe() domyślnie pokazana)
stats.loc['median'] = df[cols].median()
stats = stats.round(0).astype(int)
print(stats)

        area_m2  price_total_zl_cleaned  price_per_m2_zl_cleaned
count       992                     992                      992
mean         57                  921066                    16131
std          25                  567261                     5707
min          16                  125000                     3788
25%          42                  590000                    12149
50%          53                  767000                    14799
75%          67                 1100000                    18965
max         448                 7850000                    62800
median       53                  767000                    14799


In [7]:
# Wyszło jakieś mieszkanie o powierzchni 448 m² - sprawdzam które to
large_apartments = df[df['area_m2'] > 400]
large_apartments.head()

,url,date_posted,photos,locality,street,property_type,rooms,area_m2,owner_direct,price_total_zl,price_per_m2_zl,city,price_total_zl_cleaned,price_per_m2_zl_cleaned,area_per_room
638,https://adresowo.pl/o/mieszkanie-gdynia-orlowo...,ponad miesiąc temu,25.0,Gdynia Orłowo,ul. Świerkowa,Apartment,20.0,448.0,Oferta biura nieruchomości,4499000,10042.0,Gdynia,4499000.0,10042.0,22.4


In [8]:
# Chcę podejrzeć url tego mieszkania
df.loc[638, 'url']

'https://adresowo.pl/o/mieszkanie-gdynia-orlowo-ul-swierkowa-20-pokojowe-e1s0s7'

In [9]:
# Wyszło jakieś mieszkanie o cenie ponad 7 mn zł - sprawdzam które to
large_apartments = df[df['price_total_zl_cleaned'] > 7000000]
large_apartments.head()

,url,date_posted,photos,locality,street,property_type,rooms,area_m2,owner_direct,price_total_zl,price_per_m2_zl,city,price_total_zl_cleaned,price_per_m2_zl_cleaned,area_per_room
862,https://adresowo.pl/o/mieszkanie-gdynia-srodmi...,ponad miesiąc temu,20.0,Gdynia Śródmieście,ul. A. Hryniewickiego,Apartment,3.0,125.0,Bez pośredników,7850000,62800.0,Gdynia,7850000.0,62800.0,41.666667


In [10]:
# Chcę podejrzeć url tego mieszkania
df.loc[862, 'url']

'https://adresowo.pl/o/mieszkanie-gdynia-srodmiescie-ul-a-hryniewickiego-3-pokojowe-g1k0e7'

In [11]:
# Grupuję po mieście i obliczam średnią oraz liczbę ogłoszeń
city_stats = (
    df.groupby('city')
    .agg({
        'area_m2': 'mean',
        'price_total_zl_cleaned': 'mean',
        'price_per_m2_zl_cleaned': 'mean',
        'city': 'count'
    })
    .rename(columns={'city': 'count'})  # zmieniam nazwę kolumny na 'count'
    .round(0)
    .astype(int)
)

print("Statystyki według miasta - średnie (mean):")
print("-------------------------")
print(city_stats)

Statystyki według miasta - średnie (mean):
-------------------------
        area_m2  price_total_zl_cleaned  price_per_m2_zl_cleaned  count
city                                                                   
Gdańsk       55                  913870                    16661    664
Gdynia       60                  865968                    14192    269
Sopot        66                 1253257                    19011     59


In [12]:
city_counts = df['city'].value_counts().sort_values(ascending=False)
print(city_counts)

city
Gdańsk    664
Gdynia    269
Sopot      59
Name: count, dtype: int64


In [13]:
# Przedstaw dane na wykresach:
#   histogram powierzchni mieszkań,
#   boxplot cen w dzielnicach,
#   scatter plot: area_m2 vs price_total_zl,
#   ranking dzielnic wg ceny za m² (bar chart lub pie chart).
# Użyj Plotly i zapisz wybrane wykresy do PDF przy użyciu kaleido

In [14]:
px.histogram(df, x='area_m2', title='Histogram metrażu mieszkań w Trójmieście', nbins=100)

In [15]:
# Sprawdzę, czy są outlier'y dla cen mieszkań
px.histogram(df, x='price_total_zl_cleaned', title='Histogram cen mieszkań w Trójmieście', nbins=100)

In [16]:
# Usuwam dwa mieszkania outlier'y powyej 160 m2 i powyej 3,5 mn zł ponawiam wykresy
df_filtered = df[(df['area_m2'] <= 160) & (df['price_total_zl_cleaned'] <= 3_500_000)]
px.histogram(df_filtered, x='area_m2', title='Histogram metrażu mieszkań w Trójmieście (po usunięciu outlierów)', nbins=50)

In [17]:
px.histogram(df_filtered, x='price_total_zl_cleaned', title='Histogram cen mieszkań w Trójmieście (po usunięciu outlierów)', nbins=100)

In [18]:
fig = px.box(
    df_filtered,
    x='locality',
    y='price_total_zl_cleaned',
    title='Rozkład cen na podstawie dzielnic w Trójmieście (bez outlierów)',
    hover_data=['city'],
    width=1400,
    height=700,
    template='plotly_white'
)
fig.update_layout(xaxis_tickangle=-45)

fig.show()
pio.write_image(fig, "box_plot_rozkładu_cen.pdf")

In [19]:
# Widzę, ze są dzielnice z dużym rozstrzałem cenowym, np. Sopot, gdynia Orłowo, Gdańsk Brzeźno

In [20]:
fig = px.scatter(
    df_filtered,
    x='area_m2',
    y='price_total_zl_cleaned',
    color='city',
    symbol='city',
    title='Zależność między powierzchnią a ceną w Trójmieście (bez outlierów)',
    hover_data=['locality'],
    width=1400,
    height=700,
    opacity=0.7,
    template='plotly_white'
)
fig.show()
pio.write_image(fig, "powierzchnia_a_cena.pdf")

In [21]:
# Na podstawie tego wykresu widać, e ceny mieszkań w Gdyni są generalnie nisze ni w Gdańsku i Sopocie - w szczególności dotyczy to mniejszych mieszkań. PPrzy dużym metrażu różnice się zacierają.

In [22]:
average_price_per_m2_sorted = df.groupby('locality')['price_per_m2_zl_cleaned'].mean().sort_values(ascending=True)

fig = px.bar(
    x=average_price_per_m2_sorted.values,
    y=average_price_per_m2_sorted.index,
    orientation='h',
    title='Ranking dzielnic Trójmiasta według średniej ceny za m²',
    labels={'x': 'Średnia cena za m² (zł)', 'y': 'Lokalizacja'},
    color=average_price_per_m2_sorted.values,
    color_continuous_scale='Blues',
    width=1000,
    height=1600,  # zwiększamy wysokość, by zmieściły się wszystkie etykiety
    template='plotly_white'
)

# wymuszam pokazanie wszystkich nazw dzielnic
fig.update_yaxes(tickmode='linear')

fig.show()
pio.write_image(fig, "ranking_dzielnic_po_cenie.pdf")